# Automated ML

## Import Dependencies

In [2]:
import os
import csv
from pprint import pprint
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from sklearn import datasets
import pkg_resources

import azureml.core
from azureml.core.workspace import Workspace
from azureml.core.experiment import Experiment
from azureml.train.automl import AutoMLConfig
from azureml.core.compute import ComputeTarget, AmlCompute

from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn

# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

SDK version: 1.20.0


### 2. Initialize Workspace

In [3]:
ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')

quick-starts-ws-137412
aml-quickstarts-137412
southcentralus
976ee174-3882-4721-b90a-b5fef6b72f24


### 3. Initialize Experiment

In [4]:
ws = Workspace.from_config()
experiment_name = 'auto-experiment'
experiment=Experiment(ws, experiment_name)
experiment

Name,Workspace,Report Page,Docs Page
auto-experiment,quick-starts-ws-137412,Link to Azure Machine Learning studio,Link to Documentation


### 4. Create Compute Cluster

In [5]:
cpu_cluster_name = "automl-cluster"
vm_size='STANDARD_D2_V2'

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print('Found existing cluster, use it.')
except:
    compute_config = AmlCompute.provisioning_configuration(vm_size=vm_size, max_nodes=4)
    compute_target = ComputeTarget.create(ws, cpu_cluster_name, compute_config)

# Can poll for a minimum number of nodes and for a specific timeout. 
# If no min node count is provided it uses the scale settings for the cluster.
compute_target.wait_for_completion(show_output=True)

Creating
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


### 5. Dataset

In [6]:
data = datasets.load_breast_cancer()
print(data.data.shape)
print(data.feature_names)
print(data.DESCR)

(569, 30)
['mean radius' 'mean texture' 'mean perimeter' 'mean area'
 'mean smoothness' 'mean compactness' 'mean concavity'
 'mean concave points' 'mean symmetry' 'mean fractal dimension'
 'radius error' 'texture error' 'perimeter error' 'area error'
 'smoothness error' 'compactness error' 'concavity error'
 'concave points error' 'symmetry error' 'fractal dimension error'
 'worst radius' 'worst texture' 'worst perimeter' 'worst area'
 'worst smoothness' 'worst compactness' 'worst concavity'
 'worst concave points' 'worst symmetry' 'worst fractal dimension']
.. _breast_cancer_dataset:

Breast cancer wisconsin (diagnostic) dataset
--------------------------------------------

**Data Set Characteristics:**

    :Number of Instances: 569

    :Number of Attributes: 30 numeric, predictive attributes and the class

    :Attribute Information:
        - radius (mean of distances from center to points on the perimeter)
        - texture (standard deviation of gray-scale values)
        - peri

In [7]:
df = pd.DataFrame(data.data, columns = data.feature_names)
df['target']=data.target
df.head(5)

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension,target
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,0
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,0
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,0
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,0
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,0


In [23]:
pd.Series(data.target).value_counts(normalize=True)

1    0.627417
0    0.372583
dtype: float64

**Our dataset is imbalanced**

In [16]:
from azureml.data.dataset_factory import TabularDatasetFactory
from azureml.data.datapath import DataPath
# Create TabularDataset using TabularDatasetFactory
def_blob_store = ws.get_default_datastore()
print("Default datastore's name: {}".format(def_blob_store.name))
data_path = DataPath(datastore=def_blob_store, path_on_datastore='datapath')
ds = TabularDatasetFactory.register_pandas_dataframe(df, name='UCI_ML_Breast_Cancer', target=data_path)

Method register_pandas_dataframe: This is an experimental method, and may change at any time.<br/>For more information, see https://aka.ms/azuremlexperimental.


Default datastore's name: workspaceblobstore
Validating arguments.
Arguments validated.
Successfully obtained datastore reference and path.
Uploading file to datapath/32422164-ea43-468a-8a4d-5d4055a032dc/
Successfully uploaded file to datastore.
Creating and registering a new dataset.
Successfully created and registered a new dataset.


## AutoML Configuration:

Our task is `classification`, Due to a bit imbalance in data we choose AUC score as a metrics

In [21]:
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task='classification',
    primary_metric='AUC_weighted',
    training_data = ds,
    label_column_name = "target",
    n_cross_validations=5,
    compute_target=compute_target)

## Run Details

In [22]:
# TODO: Submit your experiment
remote_run = experiment.submit(automl_config)

RunDetails(remote_run).show()

Running on remote.


_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

In [24]:
remote_run.wait_for_completion(show_output=True)


Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and all classes are balanced in your training data.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData

****************************************************************************************************

TYPE:         Missing feature values imputation
STATUS:       PASSED
DESCRIPTION:  No feature missing values were detected in the training data.
              Learn more about missing value imputation: https://aka.ms/AutomatedMLFeaturization

****************************************************************************************************

TYPE:         High cardinality feature detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and no high cardi

{'runId': 'AutoML_f9894bd9-8b71-45e4-a5dd-675cd4dd8972',
 'target': 'automl-cluster',
 'status': 'Completed',
 'startTimeUtc': '2021-02-06T08:38:33.30437Z',
 'endTimeUtc': '2021-02-06T09:21:55.968224Z',
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'AUC_weighted',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': '5',
  'target': 'automl-cluster',
  'AMLSettingsJsonString': '{"path":null,"name":"auto-experiment","subscription_id":"976ee174-3882-4721-b90a-b5fef6b72f24","resource_group":"aml-quickstarts-137412","workspace_name":"quick-starts-ws-137412","region":"southcentralus","compute_target":"automl-cluster","spark_service":null,"azure_service":"remote","many_models":false,"pipeline_fetch_max_batch_size":1,"iterations":1000,"primary_metric":"AUC_weighted","task_type":"classification","data_script":null,"validation_size":0.0,"n_cross_validations":5,"y_min":null,"y_max":null,"nu

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [25]:
# Retrieve and save your best automl model.
best_run, fitted_model = remote_run.get_output()
# get_metrics()
# Returns the metrics
print("Best run metrics :",best_run.get_metrics())
# get_details()
# Returns a dictionary with the details for the run
print("Best run details :",best_run.get_details())

Package:azureml-automl-runtime, training version:1.21.0, current version:1.20.0
Package:azureml-core, training version:1.21.0.post1, current version:1.20.0
Package:azureml-dataprep, training version:2.8.2, current version:2.7.3
Package:azureml-dataprep-native, training version:28.0.0, current version:27.0.0
Package:azureml-dataprep-rslex, training version:1.6.0, current version:1.5.0
Package:azureml-dataset-runtime, training version:1.21.0, current version:1.20.0
Package:azureml-defaults, training version:1.21.0, current version:1.20.0
Package:azureml-interpret, training version:1.21.0, current version:1.20.0
Package:azureml-pipeline-core, training version:1.21.0, current version:1.20.0
Package:azureml-telemetry, training version:1.21.0, current version:1.20.0
Package:azureml-train-automl-client, training version:1.21.0, current version:1.20.0
Package:azureml-train-automl-runtime, training version:1.21.0, current version:1.20.0


Best run metrics : {'recall_score_macro': 0.974330625099833, 'average_precision_score_macro': 0.9970902215909341, 'balanced_accuracy': 0.974330625099833, 'recall_score_micro': 0.9771153547585779, 'weighted_accuracy': 0.9797110378797849, 'average_precision_score_micro': 0.9968546461826463, 'precision_score_weighted': 0.9779359404650508, 'f1_score_micro': 0.9771153547585779, 'AUC_weighted': 0.9970155381802327, 'matthews_correlation': 0.9518913953492655, 'f1_score_weighted': 0.9770268298666578, 'precision_score_macro': 0.9776147576147578, 'f1_score_macro': 0.9754275367024231, 'average_precision_score_weighted': 0.9972711209378444, 'recall_score_weighted': 0.9771153547585779, 'norm_macro_recall': 0.9486612501996659, 'AUC_macro': 0.9970155381802327, 'AUC_micro': 0.9967570881704815, 'log_loss': 0.08210202266441727, 'accuracy': 0.9771153547585779, 'precision_score_micro': 0.9771153547585779, 'confusion_matrix': 'aml://artifactId/ExperimentRun/dcid.AutoML_f9894bd9-8b71-45e4-a5dd-675cd4dd8972_2

In [26]:
fitted_model._final_estimator

PreFittedSoftVotingClassifier(classification_labels=None,
                              estimators=[('13',
                                           Pipeline(memory=None,
                                                    steps=[('standardscalerwrapper',
                                                            <azureml.automl.runtime.shared.model_wrappers.StandardScalerWrapper object at 0x7fd2451615f8>),
                                                           ('sgdclassifierwrapper',
                                                            SGDClassifierWrapper(alpha=10,
                                                                                 class_weight=None,
                                                                                 eta0=0.01,
                                                                                 fit_intercept=True,
                                                                                 l1_ratio=0.7959183673469387,
         

In [28]:
print(fitted_model)

Pipeline(memory=None,
         steps=[('datatransformer',
                 DataTransformer(enable_dnn=None, enable_feature_sweeping=None,
                                 feature_sweeping_config=None,
                                 feature_sweeping_timeout=None,
                                 featurization_config=None, force_text_dnn=None,
                                 is_cross_validation=None,
                                 is_onnx_compatible=None, logger=None,
                                 observer=None, task=None, working_dir=None)),
                ('prefittedsoftvotingclassifier',...
                                                                                                    min_samples_split=0.10368421052631578,
                                                                                                    min_weight_fraction_leaf=0.0,
                                                                                                    n_estimators=10,
      

In [32]:
#Save the model
best_run.register_model(model_name = 'automl_model.pkl', model_path = './outputs/')

Model(workspace=Workspace.create(name='quick-starts-ws-137412', subscription_id='976ee174-3882-4721-b90a-b5fef6b72f24', resource_group='aml-quickstarts-137412'), name=automl_model.pkl, id=automl_model.pkl:2, version=2, tags={}, properties={})

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

TODO: In the cell below, send a request to the web service you deployed to test it.

TODO: In the cell below, print the logs of the web service and delete the service